# Tutorial 4 - Setting parameter values

In [Tutorial 1](./tutorial-1-how-to-run-a-model.ipynb) and [Tutorial 2](./tutorial-2-compare-models.ipynb), we saw how to run a PyBaMM model with all the default settings. However, PyBaMM also allows you to tweak these settings for your application. In this tutorial, we will see how to change the parameters in PyBaMM.

In [ ]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm

## Change the whole parameter set

PyBaMM has a number of in-built parameter sets (check the list [here](https://docs.pybamm.org/en/latest/source/api/parameters/parameter_sets.html)), which can be selected doing

In [ ]:
parameter_values = pybamm.ParameterValues("Chen2020")

The parameter values are stored in a dictionary-like object of class [`pybamm.ParameterValues`](https://docs.pybamm.org/en/latest/source/api/parameters/parameter_values.html). 

In [ ]:
parameter_values

and the specific values can be accessed using standard dictionary syntax:

In [ ]:
parameter_values["Electrode height [m]"]

We can also search parameter values with a given keyword using the `search` command:

In [ ]:
parameter_values.search("electrolyte")

To run a simulation with a given parameter set, we can proceed as usual but passing the parameters as the `parameter_values` keyword argument

In [ ]:
model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600])
sim.plot()

## Change individual parameters

We often want to quickly change a small number of parameter values to investigate how the behaviour or the battery changes. In such cases, we can change parameter values directly in notebook or script we are working in as we demonstrate in this section.

Parameters can either have a constant value or be a function of a model variable. These dependencies are hardcoded into the PyBaMM models. In order to check what are the specific parameters that a model requires, and whether they must be constants or they can be functions, we can call the `print_parameter_info` method.

In [ ]:
model.print_parameter_info()

The table shows all the parameters that need to be provided to the `model` (in this case DFN). For example, we see that we need to define (amongst others) the `Negative electrode thickness [m]`, which is defined to be a `Parameter`. This means it can only have a constant value. In contrast, other parameters are defined to be a `FunctionParameter`, which means that they can depend on model variables. For example, the `Current function [A]` is a `FunctionParameter` that can depend on `Time [s]`.

Note that a `FunctionParameter` can always be defined to be a constant (i.e. like if it was a `Parameter`), but a `Parameter` cannot be defined to be a function. This is because these dependencies are hardcoded in the model definitions.

### Constant parameters
Let's start with a simple example: change the `Current function [A]` to be 10 A. To do this, we can simply update the value in the dictionary:

In [ ]:
parameter_values["Current function [A]"] = 10

Note that, as we mentioned above, even though `Current function [A]` could depend on time, we can always define it to be a function. 

Now we just need to run the simulation with the new parameter values

In [ ]:
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600])
sim.plot()

and we observe in the plot that now the applied current is indeed 10 A. Note that we still passed the interval `[0, 3600]` to `sim.solve()`, but the simulation terminated early as the lower voltage cut-off was reached.

### Function parameters

Let's now illustrate how to change the `Current function [A]` to be a given time-dependent function. In this case we will set the current to be sinusoidal. To do this, we need to define the relevant function and pass it as a parameter.

In [ ]:
import numpy as np


def my_current(t):
    return pybamm.sin(2 * np.pi * t / 60)


parameter_values["Current function [A]"] = my_current

Note that the `my_current` function takes an argument `t`, which is time. PyBaMM will assume that the arguments are defined exactly as in the `print_parameter_info` table above, so the actual name given to the arguments does not matter, but the order does. For example, for the `Electrolyte conductivity [S.m-1]`, it will assume that the first argument is `Electrolyte concentration [mol.m-3]` and the second is `Temperature [K]`.

We can now solve the model again. In this case, we will pass `t_eval` to be an array of the points we want to evaluate our solution to the solver (rather than the start and end points only) to make sure we have enough time points to resolve the sinusoidal function in our output.

In [ ]:
sim = pybamm.Simulation(model, parameter_values=parameter_values)
t_eval = np.arange(0, 121, 1)
sim.solve(t_eval=t_eval)
sim.plot(["Current [A]", "Voltage [V]"])

## Input parameters

If the value of a parameter is expected to change often (e.g. running a parameter sweep) is is more convenient to set a parameter as an "input parameter". This is a placeholder that can be filled in with a numerical value when the model is solved.

To set a parameter as an input parameter, we can set its value to the string `[input]` in the parameter values dictionary. For example, we can set the `Current function [A]` to be an input parameter and then run a parameter sweep over different current values like so:

In [ ]:
import matplotlib.pyplot as plt

parameter_values["Current function [A]"] = "[input]"
sim = pybamm.Simulation(model, parameter_values=parameter_values)
solns = []
for c in [0.1, 0.2, 0.3]:
    soln = sim.solve([0, 3600], inputs={"Current function [A]": c})
    plt.plot(soln["Time [s]"].entries, soln["Voltage [V]"].entries, label=f"{c} A")
    solns.append(soln["Terminal voltage [V]"].entries)
plt.xlabel("Time [s]")
plt.ylabel("Terminal voltage [V]")
plt.legend()
plt.show()

## Define a new parameter set

We can also define a new parameter set from scratch, which is useful if there is a new battery or chemistry on which we need to run simulations repeatedly. To do so, we can initialise a `ParameterValues` object and pass as an argument a dictionary of parameter values:

In [ ]:
def cube(t):
    return t**3


parameter_values = pybamm.ParameterValues(
    {
        "Negative electrode thickness [m]": 1e-4,
        "Positive electrode thickness [m]": 1.2e-4,
        "Current function [A]": cube,
    }
)

Note how, when we pass a function as a parameter, we pass the object without calling it, i.e. we pass `cube` rather than `cube(t)`. This new `parameter_values` variable could now be passed to a simulation, but note that it is incomplete as it does not include all the parameters that the model needs to run (see the parameters needed by calling `model.print_parameter_info()`, as done above). 

It is often convenient to define the parameter set in a separate file, and then call the parameters into your notebook or script. You can find some examples on how to do so in [PyBaMM's parameter library](https://github.com/pybamm-team/PyBaMM/tree/develop/src/pybamm/input/parameters/lithium_ion). You can copy one of the parameter sets available into a new file and modify it accordingly for the new parameter set. Then, whenever the set is needed, one can import the `get_parameter_values` method from the corresponding file and call it to obtain a copy of the parameter values.

In this notebook we have seen how we can change the parameters of our model. In [Tutorial 5](./tutorial-5-run-experiments.ipynb) we show how can we define and run experiments.

## References

The relevant papers for this notebook are:

In [ ]:
pybamm.print_citations()